In [36]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.models import Model
from keras.layers import Dense
from urllib import request
import tempfile

In [37]:
root_dir = './data/kaggle/dog-breed/'
labels_path = os.path.join(root_dir, 'labels.csv')
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

In [40]:
#
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
    zip_path = os.path.join(tempfile.gettempdir(), 'dog-breed.zip')
    print('Start download!')
    request.urlretrieve('https://kienle.blob.core.windows.net/public/kaggle/dog-breed.zip', zip_path)
    print('Start unzip')
    import zipfile
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(root_dir)
    with zipfile.ZipFile(os.path.join(root_dir,'test.zip'), 'r') as zip_ref:
        zip_ref.extractall(root_dir)
    with zipfile.ZipFile(os.path.join(root_dir,'train.zip'), 'r') as zip_ref:
        zip_ref.extractall(root_dir)
    os.remove(zip_path)
    os.remove(os.path.join(root_dir,'train.zip'))
    os.remove(os.path.join(root_dir,'test.zip'))
    print('Done')

Start download!
Start unzip
Done


In [41]:
!ls $root_dir

labels.csv            test                  train
sample_submission.csv test.zip              train.zip


In [10]:
gen = ImageDataGenerator()

In [11]:
pd_images = pd.read_csv(labels_path)

In [14]:
pd_images.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [18]:
images_generator = gen.flow_from_dataframe(pd_images, train_dir, x_col='id',  y_col='breed', has_ext=False)

Found 10222 images belonging to 120 classes.


In [27]:
num_classes = len(images_generator.class_indices)
num_classes

120

In [28]:
## Build model
resnetModel = ResNet50()

In [21]:
resnetModel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [31]:
outputs = Dense(num_classes)(resnetModel.get_layer('avg_pool').output)
model = Model(inputs = resnetModel.input, outputs = outputs)

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation